# Pad to the largest image size in dataset

In [3]:
import re
import numpy as np
from skimage.io import imread, imsave
from pathlib import Path

in_dir = Path(r"C:/Users/Data Analysis/Desktop/project_trainingAggression/neuropixelsProbePlacementValidation/Registration/ImagesForRegistration/Tiff_DAPI_DownsampledRenamed")
out_dir = Path(r"C:/Users/Data Analysis/Desktop/project_trainingAggression/neuropixelsProbePlacementValidation/Registration/ImagesForRegistration/paddedImages")
out_dir.mkdir(exist_ok=True)

# ---- helpers ----
trailing_id = re.compile(r"^(?P<base>.*?)(?:[_-]?)(?P<id>\d+)$")

def split_base_and_id(p: Path):
    m = trailing_id.match(p.stem)
    if not m:
        raise ValueError(f"No trailing numeric ID in filename: {p.name}")
    base = m.group("base")
    # clean any leftover separators at the end of base
    base = re.sub(r"[_\-\s\.]+$", "", base)
    id_str = m.group("id")  # keep zero-padding as-is
    return base, id_str

# sort by numeric value of the trailing ID
paths = sorted([*in_dir.glob("*.tif"), *in_dir.glob("*.tiff")],
               key=lambda p: int(split_base_and_id(p)[1]))

# load to find max H/W for padding
imgs = [imread(p) for p in paths]
H = max(im.shape[0] for im in imgs)
W = max(im.shape[1] for im in imgs)

def pad_to(im, H, W):
    h, w = im.shape[:2]
    pt = (H - h) // 2
    pb = H - h - pt
    pl = (W - w) // 2
    pr = W - w - pl
    pads = ((pt, pb), (pl, pr)) if im.ndim == 2 else ((pt, pb), (pl, pr), (0, 0))
    return np.pad(im, pads, mode="constant")

# save with ID moved to the front
for p, im in zip(paths, imgs):
    im_padded = pad_to(im, H, W)
    base, id_str = split_base_and_id(p)
    new_name = f"{id_str}_{base}{p.suffix.lower()}"
    imsave(out_dir / new_name, im_padded)

c:\Users\Data Analysis\anaconda3\envs\ProbeTrajectoryReg\Lib\site-packages\skimage\_shared\utils.py:328: UserWarning: C:\Users\Data Analysis\Desktop\project_trainingAggression\neuropixelsProbePlacementValidation\Registration\ImagesForRegistration\paddedImages\001_20250716_LSs1S1_Dapi_10x_Tile_948298-Downsampled_section.tif is a low contrast image
  return func(*args, **kwargs)
c:\Users\Data Analysis\anaconda3\envs\ProbeTrajectoryReg\Lib\site-packages\skimage\_shared\utils.py:328: UserWarning: C:\Users\Data Analysis\Desktop\project_trainingAggression\neuropixelsProbePlacementValidation\Registration\ImagesForRegistration\paddedImages\002_20250716_LSs2S1_Dapi_10x_Tile_948298-Downsampled_section.tif is a low contrast image
  return func(*args, **kwargs)
c:\Users\Data Analysis\anaconda3\envs\ProbeTrajectoryReg\Lib\site-packages\skimage\_shared\utils.py:328: UserWarning: C:\Users\Data Analysis\Desktop\project_trainingAggression\neuropixelsProbePlacementValidation\Registration\ImagesForRegist